# 取次店未清算合計集計（例）

* 自分のGoogle Driveにある　「取次店手数料_2021年10月から2022年10月まで.xlsx」　ファイルの　「Sheet1]　シートを　「df1」　に読み込む
* 自分のGoogle Driveにある　「取次店マスタ_202210.xlsx」　ファイルの　「Sheet1]　シートを　「df2」　に読み込む
* 「df1」の　'清算'カラムを　'未清算'にフィルターし、　'取次店番号', '取次店番号（サブ）'でピボット（GroupBy)し、'取次店手数料（円）'の合計を出す
* ピボットしたindex（'取次店番号', '取次店番号（サブ）'）にVLOOKUPをかけるため、indexをカラムに変換し、変換したものを　「df3」　に入れる
* 「df3」　に対して　「df2」　の同じカラムを基準にVLOOKUPする（merge）
  * その結果を　'取次店手数料（円）'　が大きい順に並び替える
  * 並び替えた結果を　「df4」　に入れる
* 「df4」　を　「未清算取次店金額一蘭_20211001-20221012.xlsx」　に保存する

In [29]:
import pandas as pd

In [30]:
df1 = pd.read_excel('drive/MyDrive/取次店手数料_2021年10月から2022年10月まで.xlsx', sheet_name='Sheet1')

In [31]:
df2 = pd.read_excel('drive/MyDrive/取次店マスタ_202210.xlsx', sheet_name='Sheet1')

In [32]:
df1

,年月日,お客様番号,取次店番号,取次店番号（サブ）,取次店手数料（円）,清算
0,2021-10-01,17228,151,5,-10000,済み
1,2021-10-01,11828,170,5,0,済み
2,2021-10-01,17602,103,7,180000,済み
3,2021-10-01,10564,104,5,160000,済み
4,2021-10-01,13421,173,6,270000,済み
...,...,...,...,...,...,...
251152,2022-10-12,18434,119,1,175000,未清算
251153,2022-10-12,14381,174,2,180000,済み
251154,2022-10-12,14508,132,7,30000,済み
251155,2022-10-12,15214,112,3,100000,済み


In [33]:
df1[df1['清算']=='未清算']

,年月日,お客様番号,取次店番号,取次店番号（サブ）,取次店手数料（円）,清算
7,2021-10-01,19854,137,7,-45000,未清算
8,2021-10-01,19754,135,5,-45000,未清算
10,2021-10-01,12231,103,4,-35000,未清算
11,2021-10-01,13800,157,3,25000,未清算
18,2021-10-01,15122,110,5,30000,未清算
...,...,...,...,...,...,...
251115,2022-10-12,14127,172,1,120000,未清算
251126,2022-10-12,18942,142,2,15000,未清算
251134,2022-10-12,17055,118,2,20000,未清算
251139,2022-10-12,19644,150,3,120000,未清算


In [34]:
df1[df1['清算']=='未清算'][['取次店番号','取次店番号（サブ）','取次店手数料（円）']].groupby(['取次店番号', '取次店番号（サブ）']).sum('取次店手数料（円）')

取次店手数料（円）
取次店番号 取次店番号（サブ）           
100   1            1635000
      2            2205000
      3            1920000
      4            2810000
      5            1525000
...                    ...
179   3            2355000
      4            2010000
      5            3640000
      6            3750000
      7            2825000

[560 rows x 1 columns]

In [35]:
df1[df1['清算']=='未清算'][['取次店番号','取次店番号（サブ）','取次店手数料（円）']].groupby(['取次店番号', '取次店番号（サブ）']).sum('取次店手数料（円）').reset_index()

,取次店番号,取次店番号（サブ）,取次店手数料（円）
0,100,1,1635000
1,100,2,2205000
2,100,3,1920000
3,100,4,2810000
4,100,5,1525000
...,...,...,...
555,179,3,2355000
556,179,4,2010000
557,179,5,3640000
558,179,6,3750000


In [36]:
df3 = df1[df1['清算']=='未清算'][['取次店番号','取次店番号（サブ）','取次店手数料（円）']].groupby(['取次店番号', '取次店番号（サブ）']).sum('取次店手数料（円）').reset_index()

In [37]:
df2

,取次店番号,取次店番号（サブ）,取次店名,支店名,住所,担当者,会社メール
0,151,5,合同会社山口印刷,中央支店,沖縄県府中市浅草橋31丁目4番8号 今戸コーポ909,池田 舞,tsubasamatsuda@yoshida.com
1,170,5,合同会社藤原農林,川崎市川崎支店,石川県川崎市幸区氏家新田37丁目1番16号 アーバン睦637,太田 太一,taroaoki@matsumoto.com
2,103,7,池田鉱業合同会社,台東支店,徳島県香取郡東庄町木立26丁目7番15号,斎藤 晃,momoko83@nishimura.jp
3,104,5,株式会社石川印刷,多摩支店,宮崎県文京区千塚18丁目16番19号 コーポ西小来川622,林 和也,nanamiogawa@goto.jp
4,173,6,合同会社石川印刷,西多摩郡日の出支店,福島県印旛郡本埜村池之端16丁目1番18号 シャルム千塚163,高橋 春香,pnakagawa@sato.jp
...,...,...,...,...,...,...,...
555,161,1,株式会社加藤食品,本店,群馬県神津島村一ツ橋18丁目7番12号 西浅草シャルム604,小川 太郎,ryosukeshimizu@sato.com
556,159,2,斎藤電気有限会社,西多摩郡日の出支店,神奈川県東村山市前弥六南町11丁目24番13号,岡田 真綾,manabu86@fukuda.jp
557,124,4,井上建設合同会社,多摩支店,愛媛県福生市二つ室24丁目13番14号 ハイツ白金599,松田 陽子,miturumurakami@kimura.jp
558,132,6,有限会社村上農林,安房郡鋸南支店,北海道山武市木立22丁目20番2号 隼町シャルム829,斎藤 知実,qyamazaki@sasaki.com


In [38]:
df3.merge(df2, how='left')

,取次店番号,取次店番号（サブ）,取次店手数料（円）,取次店名,支店名,住所,担当者,会社メール
0,100,1,1635000,前田鉱業有限会社,本店,大分県西多摩郡瑞穂町小入30丁目10番18号 東三島アーバン282,岡田 智也,yumiko71@nakajima.jp
1,100,2,2205000,有限会社藤原印刷,八王子支店,岩手県稲城市松浦町10丁目17番6号,山崎 七夏,naokifujii@shimizu.jp
2,100,3,1920000,有限会社西村保険,山武郡芝山支店,群馬県川崎市多摩区中川崎3丁目17番15号 白金台クレスト071,小林 零,bfujiwara@aoki.jp
3,100,4,2810000,中島ガス株式会社,山武郡芝山支店,沖縄県山武郡九十九里町湯宮2丁目9番14号 千塚コート461,清水 桃子,ishiikazuya@okada.net
4,100,5,1525000,近藤印刷株式会社,横浜市都筑支店,京都府狛江市浅草橋24丁目18番19号,坂本 知実,akiraishii@miura.net
...,...,...,...,...,...,...,...,...
555,179,3,2355000,吉田食品株式会社,青ヶ島支店,宮崎県鴨川市羽折町2丁目18番19号 東神田コート031,石井 裕美子,yamadayoichi@ogawa.com
556,179,4,2010000,有限会社小林銀行,浦安支店,宮崎県調布市上広谷39丁目25番10号 シャルム鳥越808,藤井 陽一,naoki40@takahashi.jp
557,179,5,3640000,清水電気合同会社,清瀬支店,宮崎県横浜市戸塚区虎ノ門虎ノ門ヒルズ森タワー31丁目24番11号 南郷屋シティ907,斉藤 里佳,nakagawaminoru@yamada.com
558,179,6,3750000,田中銀行合同会社,横浜市中支店,岩手県練馬区平河町24丁目6番3号,長谷川 学,jyamashita@aoki.net


In [39]:
df3.merge(df2, how='left').sort_values('取次店手数料（円）', ascending=False)

,取次店番号,取次店番号（サブ）,取次店手数料（円）,取次店名,支店名,住所,担当者,会社メール
310,144,3,4115000,合同会社斉藤保険,足立支店,鹿児島県横浜市旭区丸の内ＪＰタワー27丁目19番1号 九段南シャルム740,藤田 知実,atsushi84@yamaguchi.com
316,145,2,3960000,岡田電気有限会社,印西支店,広島県富里市丹勢39丁目4番2号,山本 学,rnakagawa@maeda.net
26,103,6,3950000,合同会社坂本通信,横浜市青葉支店,三重県港区脚折町19丁目8番4号 京橋コーポ412,田中 零,fishikawa@inoue.org
529,175,5,3905000,合同会社加藤運輸,江東支店,徳島県長生郡長生村西浅草11丁目9番17号 日光パーク302,橋本 結衣,tomoyamori@goto.org
72,110,3,3890000,有限会社山本銀行,山武郡横芝光支店,滋賀県西東京市松浦町31丁目10番19号,岡本 真綾,hmiura@kondo.jp
...,...,...,...,...,...,...,...,...
335,147,7,920000,合同会社斉藤鉱業,川崎市宮前支店,千葉県横浜市南区東神田8丁目17番12号,木村 太一,maayasakamoto@abe.jp
76,110,7,870000,株式会社加藤電気,渋谷支店,千葉県国分寺市芝浦9丁目2番4号 吾妻橋アーバン103,斎藤 聡太郎,kenichi05@sasaki.jp
304,143,4,855000,合同会社石川銀行,長生郡長生支店,福島県西多摩郡奥多摩町浅草橋39丁目2番2号 パレス白金台771,阿部 裕樹,tomoya74@yamazaki.jp
138,119,6,835000,太田通信合同会社,八丈島八丈支店,佐賀県印西市日光9丁目23番16号 パレス皇居外苑515,佐々木 聡太郎,ryamashita@yoshida.jp


In [40]:
df4 = df3.merge(df2, how='left').sort_values('取次店手数料（円）', ascending=False)

In [41]:
filename = '未清算取次店金額一蘭_20211001-20221012.xlsx'

In [43]:
df4.to_excel(f'drive/MyDrive/{filename}', index=False)

# 結果をメールに送信する

In [44]:
# https://medium.com/@simonprdhm/how-to-send-emails-with-gmail-using-python-f4a8bcb6a9cc

# Import Python Packages
import smtplib

# Set Global Variables
gmail_user = '@gmail.com'   # 自分のgmailアカウント
gmail_password = ''         # 自分おgmailアカウントのパスワード

# Create Email 
mail_from = gmail_user
mail_to = 'sungh.kim@qcells.com'
mail_subject = f'[作成完了]{filename}'
mail_message_body = df4.to_html()

mail_message = f'''\
From: {mail_from}
To: {mail_to}
Subject: {mail_subject}
{mail_message_body}
'''

# Sent Email
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(gmail_user, gmail_password)
server.sendmail(mail_from, mail_to, mail_message)
server.close()

SMTPAuthenticationError: ignored